## pymongo 操作

http://www.jb51.net/article/77537.htm


http://www.jb51.net/article/64991.htm


In [2]:
from pymongo import MongoClient
from bson.objectid import ObjectId

# 测试本地环境下mongodb
c = MongoClient()
print('连接状态：',c.test_database)
# Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database')

连接状态： Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database')


### 基本操作 
> 连接，查询数据库，连接数据库/集合

* 连接：MongoClient(host, port)
* 查询有多少数据库：client.database_names()
* 连接到某个数据库：client.databaseName
* 查询某个数据库下有多少个集合：client.databaseName.collection_name()

In [3]:
#建立于MongoClient 的连接
client = MongoClient('localhost', 27017)

# 查看有多少数据库
dbs = client.database_names()
print('\n数据库有：', dbs)

# 连接到某个数据库
db = client.hiso # client['hiso']
print('\n现在的数据库名：', db.name)

col_names = db.collection_names()
print('\n该数据库下的集合：', col_names)

# 查看连接信息
print(db.client.address)


数据库有： ['admin', 'hiso', 'local', 'ten_aistock', 'ye_payment', 'ye_subscribe', 'ye_user']

现在的数据库名： hiso

该数据库下的集合： ['user', 'tt']
('localhost', 27017)


### 连接服务器上的mongoDB 带权限

http://api.mongodb.com/python/current/examples/authentication.html 

In [ ]:
import pymongo
from pymongo import MongoClient

# mongoDB服务器ip
host = '127.0.0.1'
# mongoDB端口
port = 27017
# 
username = 'wills'
pw = 'wills1234'

# 方法一
# client_one = MongoClient("mongodb://" + username + ":" + pw + "@" + host + '/admin')

# 方法二
client = MongoClient(host, port)
# 用户名密码连接
client.admin.authenticate(username,pw,mechanism="MONGODB-CR")

# 查看该mongoDB上有多少databases
print(client.database_names())


#### 写入/插入数据
* 单条数据插入 insert_one()
* 多条数据插入 insert_many()


In [4]:
# 连接到某个collection
col_user = db.user

# 单条插入数据
rs = col_user.insert_one({'username': 'willschang', 'age': 21, 'email': 'test@134.com'})
print('\n插入数据_id：', rs.inserted_id)

# 多条数据写入
insert_data = [
    dict(
        username='kris',
        age=22,
        email='te1@qq.com'
    ),
    dict(
        username='april',
        age=23,
        email='te2@qq.com'
    )
]
rs_many = col_user.insert_many(insert_data)
print(rs_many.inserted_ids)


插入数据_id： 594228e30d5b981a5519f548
[ObjectId('594228e30d5b981a5519f549'), ObjectId('594228e30d5b981a5519f54a')]


#### 查询数据
* 查询单条数据 find_one(query, )
* 查询多条数据 find()

In [11]:
# 查询单条数据
user = col_user.find_one({'username':'willschang'})
print('查询到单条数据为：', user)

# 查询多第数据
users = col_user.find({'username':'willschang', 'age':21})
print('查询多条数据：')
for u in users:
    print(u)

查询到单条数据为： {'_id': ObjectId('5937b2070d5b984f27e69219'), 'username': 'willschang'}
查询多条数据：
{'_id': ObjectId('5937c0f50d5b984fb2c8e2bd'), 'age': 21, 'username': 'willschang', 'email': 'test@134.com'}
{'_id': ObjectId('5937c0fd0d5b984fb2c8e2be'), 'age': 21, 'username': 'willschang', 'email': 'test@134.com'}
{'_id': ObjectId('5937c1150d5b984fb2c8e2bf'), 'age': 21, 'username': 'willschang', 'email': 'test@134.com'}
{'_id': ObjectId('5937c1270d5b984fb2c8e2c0'), 'age': 21, 'username': 'willschang', 'email': 'test@134.com'}
{'_id': ObjectId('5937c13d0d5b984fb2c8e2c1'), 'age': 21, 'username': 'willschang', 'email': 'test@134.com'}
{'_id': ObjectId('5940ff770d5b9814fc5d25ef'), 'age': 21, 'username': 'willschang', 'email': 'test@134.com'}
{'_id': ObjectId('594228e30d5b981a5519f548'), 'age': 21, 'username': 'willschang', 'email': 'test@134.com'}


#### 更新数据
```
## pymongo旧版本，仅做参考
my_set.update(
   <query>,    #查询条件
   <update>,    #update的对象和一些更新的操作符
   {
     upsert: <boolean>,    #如果不存在update的记录，是否插入
     multi: <boolean>,        #可选，mongodb 默认是false,只更新找到的第一条记录
     writeConcern: <document>    #可选，抛出异常的级别。
   }
)
```

* DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.

In [20]:
# 更新单条数据
update1 = col_user.update_one({'username': 'willschang'}, {'$set': {'age': 207}})
# 更新多条数据
updates = col_user.update_many({'username': 'willschang'}, {'$set': {'age': 207}})


In [13]:
def to_dict_str(value):
    """递归序列化dict
    """
    for k, v in value.items():
        if isinstance(v, dict):
            value[k] = to_dict_str(v)
            continue

        if isinstance(v, list):
            value[k] = to_list_str(v)
            continue

        value[k] = default_encode(v)

    return value

def to_list_str(value):
    """递归序列化list
    """
    for index, v in enumerate(value):
        if isinstance(v, dict):
            value[index] = to_dict_str(v)
            continue

        if isinstance(v, list):
            value[index] = to_list_str(v)
            continue

        value[index] = default_encode(v)

    return value

def default_encode(v):
    """数据类型转换
    """
    if isinstance(v, ObjectId):
        return str(v)
    return v

data = col_user.find()

t = to_list_str(list(data))
print(t)
    


[{'_id': '5937b2070d5b984f27e69219', 'username': 'willschang'}, {'_id': '5937b3440d5b984f27e6921c', 'username': 'willschang'}, {'_id': '5937c0f50d5b984fb2c8e2bd', 'email': 'test@134.com', 'age': 21, 'username': 'willschang'}, {'_id': '5937c0fd0d5b984fb2c8e2be', 'email': 'test@134.com', 'age': 21, 'username': 'willschang'}, {'_id': '5937c1150d5b984fb2c8e2bf', 'email': 'test@134.com', 'age': 21, 'username': 'willschang'}, {'_id': '5937c1270d5b984fb2c8e2c0', 'email': 'test@134.com', 'age': 21, 'username': 'willschang'}, {'_id': '5937c13d0d5b984fb2c8e2c1', 'email': 'test@134.com', 'age': 21, 'username': 'willschang'}, {'_id': '5937c28e0d5b984fb2c8e2c2', 'email': 'te1@qq.com', 'age': 22, 'username': 'kris'}, {'_id': '5937c28e0d5b984fb2c8e2c3', 'email': 'te2@qq.com', 'age': 23, 'username': 'april'}, {'_id': '5937c2b70d5b984fb2c8e2c4', 'email': 'te1@qq.com', 'age': 22, 'username': 'kris'}, {'_id': '5937c2b70d5b984fb2c8e2c5', 'email': 'te2@qq.com', 'age': 23, 'username': 'april'}, {'_id': '594